In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

In [4]:
log_dir = 'log/20210906_172020_test_prior_m14_copy/'
data = load_obj('data', log_dir)


In [5]:
data.raw_data

{'random_seed': 0,
 'N': 200,
 'K': 2,
 'J': 6,
 'alpha': array([0., 0., 0., 0., 0., 0.]),
 'beta': array([[1. , 0. ],
        [0.8, 0. ],
        [0.8, 0. ],
        [0. , 1. ],
        [0. , 0.8],
        [0. , 0.8]]),
 'sigma_z': array([0.80622577, 0.80622577]),
 'Phi_corr': array([[1. , 0.2],
        [0.2, 1. ]]),
 'Phi_cov': array([[0.65, 0.13],
        [0.13, 0.65]]),
 'Marg_cov': array([[1.    , 0.52  , 0.52  , 0.13  , 0.104 , 0.104 ],
        [0.52  , 1.    , 0.416 , 0.104 , 0.0832, 0.0832],
        [0.52  , 0.416 , 1.    , 0.104 , 0.0832, 0.0832],
        [0.13  , 0.104 , 0.104 , 1.    , 0.52  , 0.52  ],
        [0.104 , 0.0832, 0.0832, 0.52  , 1.    , 0.416 ],
        [0.104 , 0.0832, 0.0832, 0.52  , 0.416 , 1.    ]]),
 'Theta': array([[0.35 , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.584, 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.584, 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.35 , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.584, 

## Run MCMC

In [6]:
posterior_samples = dict()


In [8]:
## model 1 
model_num = 7

sm = load_obj('sm', 'log/compiled_models/model'+str(model_num)+'/')

param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
stan_names = model_phonebook(model_num)['stan_names']

fit_run = run_mcmc(
    data.get_stan_data(),
    sm,
    num_samples = 1000,
    num_warmup = 1000,
    adapt_engaged=True,
    num_chains = 1,
    log_dir = './log/20210906_172026_test_prior_m7_copy/'
)
ps = fit_run.extract(permuted=False, pars=param_names)
posterior_samples['batch1'] = ps

In [9]:
## model 2 
model_num = 14

sm = load_obj('sm', 'log/compiled_models/model'+str(model_num)+'/')

param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
stan_names = model_phonebook(model_num)['stan_names']

fit_run = run_mcmc(
    data.get_stan_data(),
    sm,
    num_samples = 1000,
    num_warmup = 1000,
    adapt_engaged=True,
    num_chains = 1,
    log_dir = './log/20210906_172020_test_prior_m14_copy/'
)
ps = fit_run.extract(permuted=False, pars=param_names)
posterior_samples['batch2'] = ps

## Plot Both results

In [20]:
param = 'beta'
df = get_post_df(np.squeeze(posterior_samples['batch1'][param]))
df['source'] = 'data-dependent-prior'

df2 = get_post_df(np.squeeze(posterior_samples['batch2'][param]))
df2['source'] = 'independent-prior'

plot_density(pd.concat([df, df2]))

alt.Chart(...)

In [21]:
param = 'alpha'
df = get_post_df(np.squeeze(posterior_samples['batch1'][param]))
df['source'] = 'data-dependent-prior'

df2 = get_post_df(np.squeeze(posterior_samples['batch2'][param]))
df2['source'] = 'independent-prior'

plot_density(pd.concat([df, df2]))

alt.Chart(...)